# Requirements

In [ ]:
!pip install scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import networkx as nx

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import mean_absolute_error

from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon

# Preprocessing

## MatrixVectorizer

In [ ]:
class MatrixVectorizer:
    """
    A class for transforming between matrices and vector representations.

    This class provides methods to convert a symmetric matrix into a vector (vectorize)
    and to reconstruct the matrix from its vector form (anti_vectorize), focusing on
    vertical (column-based) traversal and handling of elements.
    """

    def __init__(self):
        """
        Initializes the MatrixVectorizer instance.

        The constructor currently does not perform any actions but is included for
        potential future extensions where initialization parameters might be required.
        """
        pass

    @staticmethod
    def vectorize(matrix, include_diagonal=False):
        """
        Converts a matrix into a vector by vertically extracting elements.

        This method traverses the matrix column by column, collecting elements from the
        upper triangle, and optionally includes the diagonal elements immediately below
        the main diagonal based on the include_diagonal flag.

        Parameters:
        - matrix (numpy.ndarray): The matrix to be vectorized.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the vectorization.
          Defaults to False.

        Returns:
        - numpy.ndarray: The vectorized form of the matrix.
        """
        # Determine the size of the matrix based on its first dimension
        matrix_size = matrix.shape[0]

        # Initialize an empty list to accumulate vector elements
        vector_elements = []

        # Iterate over columns and then rows to collect the relevant elements
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Collect upper triangle elements
                        vector_elements.append(matrix[row, col])
                    elif include_diagonal and row == col + 1:
                        # Optionally include the diagonal elements immediately below the diagonal
                        vector_elements.append(matrix[row, col])

        return np.array(vector_elements)

    @staticmethod
    def anti_vectorize(vector, matrix_size, include_diagonal=False):
        """
        Reconstructs a matrix from its vector form, filling it vertically.

        The method fills the matrix by reflecting vector elements into the upper triangle
        and optionally including the diagonal elements based on the include_diagonal flag.

        Parameters:
        - vector (numpy.ndarray): The vector to be transformed into a matrix.
        - matrix_size (int): The size of the square matrix to be reconstructed.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the reconstruction.
          Defaults to False.

        Returns:
        - numpy.ndarray: The reconstructed square matrix.
        """
        # Initialize a square matrix of zeros with the specified size
        matrix = np.zeros((matrix_size, matrix_size))

        # Index to keep track of the current position in the vector
        vector_idx = 0

        # Fill the matrix by iterating over columns and then rows
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Reflect vector elements into the upper triangle and its mirror in the lower triangle
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1
                    elif include_diagonal and row == col + 1:
                        # Optionally fill the diagonal elements after completing each column
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1

        return matrix


## Pre-processing

In [ ]:
def pad_HR_adj(label, split):
  """
  Pads the adjacency matrix with zeros to match the size of the HR adjacency matrix

  Parameters:
  - label(np.array): the adjacency matrix
  - split(int): the number of zeros to pad

  Returns:
  - torch.tensor: the padded adjacency matrix
  """

  label=np.pad(label,((split,split),(split,split)),mode="constant")
  np.fill_diagonal(label,1)
  return torch.from_numpy(label).type(torch.FloatTensor)

def normalize_adj_torch(mx):
    """
    Normalize the adjacency matrix

    Parameters:
    - mx(torch.tensor): the adjacency matrix

    Returns:
    - mx(torch.tensor): the normalized adjacency matrix
    """
    rowsum = mx.sum(1)
    r_inv_sqrt = torch.pow(rowsum, -0.5).flatten()
    r_inv_sqrt[torch.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = torch.diag(r_inv_sqrt)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    mx = torch.transpose(mx, 0, 1)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    return mx

def unpad(data, split):
  """
  Unpads the data to its original size

  Parameters:
  - data(np.array): the padded data
  - split(int): the number of zeros to remove

  Returns:
  - teain(np.array): the unpadded data
  """
  idx_0 = data.shape[0]-split
  idx_1 = data.shape[1]-split
  train = data[split:idx_0, split:idx_1]
  return train

def data():
  """
  Loads the data and returns the training and testing data

  Returns:
  - train_adj(np.array): the training data (adjacency matrix)
  - train_labels(np.array): the training data (labels)
  - test_adj(np.array): the testing data (adjacency matrix)
  """

  subjects_adj_split_1 = pd.read_csv('Random-CV/Fold1/lr_split_1.csv')
  subjects_labels_split_1 = pd.read_csv('Random-CV/Fold1/hr_split_1.csv')
  subjects_adj_split_2 = pd.read_csv('Random-CV/Fold2/lr_split_2.csv')
  subjects_labels_split_2 = pd.read_csv('Random-CV/Fold2/hr_split_2.csv')
  subjects_adj_split_3 = pd.read_csv('Random-CV/Fold3/lr_split_3.csv')
  subjects_labels_split_3 = pd.read_csv('Random-CV/Fold3/hr_split_3.csv')
  test_adj = pd.read_csv('Random-CV/Fold1/lr_split_1.csv')
  test_labels = pd.read_csv('Random-CV/Fold1/lr_split_1.csv')

  # transform to numpy array
  subjects_adj_split_1 = subjects_adj_split_1.to_numpy()
  subjects_labels_split_1 = subjects_labels_split_1.to_numpy()
  subjects_adj_split_2 = subjects_adj_split_2.to_numpy()
  subjects_labels_split_2 = subjects_labels_split_2.to_numpy()
  subjects_adj_split_3 = subjects_adj_split_3.to_numpy()
  subjects_labels_split_3 = subjects_labels_split_3.to_numpy()
  test_adj = test_adj.to_numpy()
  test_labels = test_labels.to_numpy()

  # Reshape the data
  new_shape_subjects_adj_split_1 = (subjects_adj_split_1.shape[0], 160, 160)
  new_shape_subjects_labels_split_1 = (subjects_labels_split_1.shape[0], 268, 268)
  subjects_adj_3d_split_1 = np.empty(new_shape_subjects_adj_split_1)
  subjects_labels_3d_split_1 = np.empty(new_shape_subjects_labels_split_1)

  new_shape_subjects_adj_split_2 = (subjects_adj_split_2.shape[0], 160, 160)
  new_shape_subjects_labels_split_2 = (subjects_labels_split_2.shape[0], 268, 268)
  subjects_adj_3d_split_2 = np.empty(new_shape_subjects_adj_split_2)
  subjects_labels_3d_split_2 = np.empty(new_shape_subjects_labels_split_2)

  new_shape_subjects_adj_split_3 = (subjects_adj_split_3.shape[0], 160, 160)
  new_shape_subjects_labels_split_3 = (subjects_labels_split_3.shape[0], 268, 268)
  subjects_adj_3d_split_3 = np.empty(new_shape_subjects_adj_split_3)
  subjects_labels_3d_split_3 = np.empty(new_shape_subjects_labels_split_3)


  new_shape_adj = (test_adj.shape[0], 160, 160)
  test_adj_3d = np.empty(new_shape_adj)

  # Perform anti-vectorization in the loop
  for i in range(subjects_adj_split_1.shape[0]):
      # Apply anti-vectorization and reshape the i-th 1D slice to a 2D array
      subjects_adj_3d_split_1[i] = MatrixVectorizer.anti_vectorize(subjects_adj_split_1[i], 160).reshape(160, 160)
      subjects_labels_3d_split_1[i] = MatrixVectorizer.anti_vectorize(subjects_labels_split_1[i], 268).reshape(268, 268)
  for i in range(subjects_adj_split_2.shape[0]):
      # Apply anti-vectorization and reshape the i-th 1D slice to a 2D array
      subjects_adj_3d_split_2[i] = MatrixVectorizer.anti_vectorize(subjects_adj_split_2[i], 160).reshape(160, 160)
      subjects_labels_3d_split_2[i] = MatrixVectorizer.anti_vectorize(subjects_labels_split_2[i], 268).reshape(268, 268)
  for i in range(subjects_adj_split_3.shape[0]):
      # Apply anti-vectorization and reshape the i-th 1D slice to a 2D array
      subjects_adj_3d_split_3[i] = MatrixVectorizer.anti_vectorize(subjects_adj_split_3[i], 160).reshape(160, 160)
      subjects_labels_3d_split_3[i] = MatrixVectorizer.anti_vectorize(subjects_labels_split_3[i], 268).reshape(268, 268)
  for i in range(test_adj.shape[0]):
      # Apply anti-vectorization and reshape the i-th 1D slice to a 2D array
      test_adj_3d[i] = MatrixVectorizer.anti_vectorize(test_adj[i], 160).reshape(160, 160)

  train_adj_spit_1 = subjects_adj_3d_split_1
  train_labels_spit_1 = subjects_labels_3d_split_1
  train_adj_spit_2 = subjects_adj_3d_split_2
  train_labels_spit_2 = subjects_labels_3d_split_2
  train_adj_spit_3 = subjects_adj_3d_split_3
  train_labels_spit_3 = subjects_labels_3d_split_3
  test_adj = test_adj_3d

  return train_adj_spit_1, train_labels_spit_1, train_adj_spit_2, train_labels_spit_2, train_adj_spit_3, train_labels_spit_3, test_adj, None


# Model Structure

In [ ]:
# initializations(Glorot)
def weight_variable_glorot(output_dim):
    """
    Initialize weights according to Glorot initialization.

    Parameters:
    - output_dim(int): the number of output dimensions

    Returns:
    - initial(np.array): the initialized weights
    """
    input_dim = output_dim
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = np.random.uniform(-init_range, init_range,(input_dim, output_dim))

    return initial

In [ ]:
class GSRLayer(nn.Module):
  """
  The Graph Spectral Regularization layer
  """
  def __init__(self,hr_dim):
    super(GSRLayer, self).__init__()

    self.weights = torch.from_numpy(weight_variable_glorot(hr_dim)).type(torch.FloatTensor)
    self.weights = torch.nn.Parameter(data=self.weights, requires_grad = True)

  def forward(self,A,X):
    """
    Forward pass for the GSR layer

    Parameters:
    - A(torch.tensor): the adjacency matrix
    - X(torch.tensor): the input data

    Returns:
    - adj(torch.tensor): the adjacency matrix
    - torch.abs(X)(torch.tensor): the input data
    """
    lr = A
    lr_dim = lr.shape[0]
    f = X
    # Compute eigenvalues and eigenvectors
    _, U_lr = torch.linalg.eigh(A, UPLO='U')
    eye_mat = torch.eye(lr_dim).type(torch.FloatTensor).to(X.device)
    s_d = torch.cat((eye_mat,eye_mat),0)

    # Compute the GSR
    a = torch.matmul(self.weights,s_d )
    b = torch.matmul(a ,torch.t(U_lr))
    f_d = torch.matmul(b ,f)
    f_d = torch.abs(f_d)
    self.f_d = f_d.fill_diagonal_(1)
    adj = normalize_adj_torch(self.f_d)
    X = torch.mm(adj, adj.t())
    X = (X + X.t())/2
    idx = torch.eye(320, dtype=bool)
    X[idx]=1
    return adj, torch.abs(X)


class DropGNN(nn.Module):
    """
    The DropGNN layer
    """
    def __init__(self, p_dropgnn):
        super(DropGNN, self).__init__()
        self.p_dropgnn = p_dropgnn

    def forward(self, x, adj):
        """
        Forward pass for the DropGNN layer

        Parameters:
        - x(torch.tensor): the input data
        - adj(torch.tensor): the adjacency matrix

        Returns:
        - x(torch.tensor): the input data
        - adj_normalized(torch.tensor): the normalized adjacency matrix
        """
        drop = torch.bernoulli(torch.ones([x.size(0), x.size(1)], device=x.device) * self.p_dropgnn).bool()
        x[drop] = 0
        adj = adj * drop.unsqueeze(1) * drop.unsqueeze(2)

        adj = adj + torch.eye(adj.size(0)).type(torch.FloatTensor).to(x.device)
        degree = torch.sum(adj, dim=1)
        degree_sqrt_inv = torch.pow(degree, -0.5)
        degree_sqrt_inv[degree_sqrt_inv == float('inf')] = 0.
        adj_normalized = torch.matmul(torch.matmul(torch.diag(degree_sqrt_inv), adj), torch.diag(degree_sqrt_inv))
        return x, adj_normalized


class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    # 160x320 320x320 =  160x320
    def __init__(self, in_features, out_features, dropout=0., p_dropgnn=0., act=F.relu):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.p_dropgnn = p_dropgnn
        self.act = act
        self.weight = torch.nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        """
        Forward pass for the GraphConvolution layer

        Parameters:
        - input(torch.tensor): the input data
        - adj(torch.tensor): the adjacency matrix

        Returns:
        - output(torch.tensor): the output data
        """
        dropgnn = DropGNN(self.p_dropgnn)
        input, _ = dropgnn(input, adj)

        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        return output


In [ ]:
class GraphUnpool(nn.Module):
    """
    Graph Unpooling layer
    """
    def __init__(self):
        super(GraphUnpool, self).__init__()

    def forward(self, A, X, idx):
        """
        Forward pass for the GraphUnpool layer

        Parameters:
        - A(torch.tensor): the adjacency matrix
        - X(torch.tensor): the input data
        - idx(torch.tensor): the index

        Returns:
        - A(torch.tensor): the adjacency matrix
        - new_X(torch.tensor): the input data
        """
        new_X = torch.zeros([A.shape[0], X.shape[1]]).to(X.device)
        new_X[idx] = X
        return A, new_X


class GraphPool(nn.Module):
    """
    Graph Pooling layer
    """
    def __init__(self, k, in_dim, hidden_dim=64):
        super(GraphPool, self).__init__()
        self.k = k
        self.proj = nn.Linear(in_dim, 1)
        self.sigmoid = nn.Sigmoid()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, A, X):
        """
        Forward pass for the GraphPool layer

        Parameters:
        - A(torch.tensor): the adjacency matrix
        - X(torch.tensor): the input data

        Returns:
        - A(torch.tensor): the adjacency matrix
        - new_X(torch.tensor): the input data
        - idx(torch.tensor): the index
        """
        scores = self.mlp(X).squeeze()
        scores = torch.sigmoid(scores)
        num_nodes = A.shape[0]
        values, idx = torch.topk(scores, int(self.k * num_nodes))
        new_X = X[idx, :]
        values = torch.unsqueeze(values, -1)
        new_X = torch.mul(new_X, values)
        A = A[idx, :]
        A = A[:, idx]
        return A, new_X, idx


In [ ]:
class GCN(nn.Module):
    """
    The Graph Convolutional Network
    """
    def __init__(self, in_dim, out_dim):
        super(GCN, self).__init__()
        self.proj = nn.Linear(in_dim, out_dim)
        self.drop = nn.Dropout(p=0)

    def forward(self, A, X):
        """
        Forward pass for the GCN layer

        Parameters:
        - A(torch.tensor): the adjacency matrix
        - X(torch.tensor): the input data

        Returns:
        - X(torch.tensor): the input data
        """
        X = self.drop(X)
        # X = torch.matmul(A, X)
        X = self.proj(X)
        return X

In [ ]:
class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout=0., alpha=0.2, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        """
        Forward pass for the GraphAttentionLayer

        Parameters:
        - h(torch.tensor): the input data
        - adj(torch.tensor): the adjacency matrix

        Returns:
        - h_prime(torch.tensor): the output data
        """
        Wh = torch.mm(h, self.W)  # h.shape: (N, in_features), Wh.shape: (N, out_features)
        e = self._prepare_attentional_mechanism_input(Wh)

        zero_vec = - torch.inf * torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        """
        Prepare the input for the attentional mechanism
        """
        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
        # broadcast add
        e = Wh1 + Wh2.T
        return self.leakyrelu(e)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class GAT(nn.Module):
    """
    The Graph Attention Network
    """
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout

        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in
                           range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GraphAttentionLayer(nhid * nheads, nclass, dropout=dropout, alpha=alpha, concat=False)

    def forward(self, adj, x):
        """
        Forward pass for the GAT layer

        Parameters:
        - adj(torch.tensor): the adjacency matrix
        - x(torch.tensor): the input data

        Returns:
        - x(torch.tensor): the input data
        """
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj))
        return x


In [ ]:
class GraphUnet(nn.Module):
    """
    The Graph Unet
    """
    def __init__(self, ks, in_dim, out_dim, dim=320):
        super(GraphUnet, self).__init__()
        self.ks = ks

        self.start_gcn = GCN(in_dim, dim)
        self.bottom_gcn = GCN(dim, dim)
        self.end_gcn = GCN(2 * dim, out_dim)
        self.down_gcns = []
        self.up_gcns = []
        self.pools = []
        self.unpools = []
        self.l_n = len(ks)
        for i in range(self.l_n):
            # self.down_gcns.append(GCN(dim, dim))
            self.down_gcns.append(GAT(dim, dim, dim, 0., 0.2, 1))
            # self.up_gcns.append(GCN(dim, dim))
            self.up_gcns.append(GAT(dim, dim, dim, 0., 0.2, 1))
            self.pools.append(GraphPool(ks[i], dim))
            self.unpools.append(GraphUnpool())
        self.down_gcns = nn.ModuleList(self.down_gcns)
        self.up_gcns = nn.ModuleList(self.up_gcns)
        self.pools = nn.ModuleList(self.pools)
        self.unpools = nn.ModuleList(self.unpools)

    def forward(self, A, X):
        """
        Forward pass for the GraphUnet

        Parameters:
        - A(torch.tensor): the adjacency matrix
        - X(torch.tensor): the input data

        Returns:
        - X(torch.tensor): the input data
        """
        adj_ms = []
        indices_list = []
        down_outs = []
        X = self.start_gcn(A, X)
        start_gcn_outs = X
        org_X = X
        for i in range(self.l_n):
            X = self.down_gcns[i](A, X)
            adj_ms.append(A)
            down_outs.append(X)
            A, X, idx = self.pools[i](A, X)
            indices_list.append(idx)
        X = self.bottom_gcn(A, X)
        for i in range(self.l_n):
            up_idx = self.l_n - i - 1

            A, idx = adj_ms[up_idx], indices_list[up_idx]
            A, X = self.unpools[i](A, X, idx)
            X = self.up_gcns[i](A, X)
            X = X.add(down_outs[up_idx])
        X = torch.cat([X, org_X], 1)
        X = self.end_gcn(A, X)

        return X, start_gcn_outs


In [ ]:
class GSRNet(nn.Module):
    """
    The GSR Network
    """
    def __init__(self, ks, args):
        super(GSRNet, self).__init__()

        self.lr_dim = args.lr_dim
        self.hr_dim = args.hr_dim
        self.hidden_dim = args.hidden_dim
        self.layer = GSRLayer(self.hr_dim)
        self.net = GraphUnet(ks, self.lr_dim, self.hr_dim)
        self.gc1 = GraphConvolution(self.hr_dim, self.hidden_dim, 0, p_dropgnn=0., act=F.relu)
        self.gc2 = GraphConvolution(self.hidden_dim, self.hr_dim, 0, p_dropgnn=0., act=F.relu)

    def forward(self, lr):
        """
        Forward pass for the GSRNet

        Parameters:
        - lr(torch.tensor): the input data

        Returns:
        - torch.abs(z)(torch.tensor): the input data
        - net_outs(torch.tensor): the output data
        - start_gcn_outs(torch.tensor): the output data
        - outputs(torch.tensor): the output data
        """
        I = torch.eye(self.lr_dim).type(torch.FloatTensor).to(lr.device)
        A = normalize_adj_torch(lr).type(torch.FloatTensor).to(lr.device)

        self.net_outs, self.start_gcn_outs = self.net(A, I)

        self.outputs, self.Z = self.layer(A, self.net_outs)

        self.hidden1 = self.gc1(self.Z, self.outputs)
        self.hidden2 = self.gc2(self.hidden1, self.outputs)

        z = self.hidden2
        z = (z + z.t()) / 2
        idx = torch.eye(self.hr_dim, dtype=bool)
        z[idx] = 1

        return torch.abs(z), self.net_outs, self.start_gcn_outs, self.outputs


# Results Visualization

In [ ]:
def plot_loss(all_epochs_loss):
    """
    Plots the loss

    Parameters:
    - all_epochs_loss(list): the loss

    Returns:
    - None
    """
    plt.figure()
    for i in range(len(all_epochs_loss)):
        plt.plot(range(len(all_epochs_loss[i])), all_epochs_loss[i], label = f'Fold {i}')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def plot_mae(val_mae):
    """
    Plots the MAE

    Parameters:
    - val_mae(list): the MAE

    Returns:
    - None
    """
    plt.figure()
    plt.bar(range(len(val_mae)), val_mae)
    plt.xlabel('Folds')
    plt.ylabel('MAE')
    plt.title('Validation MAE')
    plt.show()

def plot_fold_eval_measures(measure_vals, fold, measure_names, ext):
    """
    Plots the evaluation measures for each fold

    Parameters:
    - measure_vals(list): the evaluation measures
    - fold(int): the fold
    - measure_names(list): the names of the evaluation measures
    - ext(str): the extension

    Returns:
    - None
    """
    x = np.arange(len(measure_names))  # the label locations
    width = 0.25  # the width of the bars

    fig, ax = plt.subplots()
    ax.bar(x, measure_vals, width, label=f'Fold {fold}', color=['red', 'blue', 'green'])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Evaluation Measures')
    ax.set_title(f'Fold {fold}')
    ax.set_xticks(x)
    ax.set_xticklabels(measure_names)

    fig.tight_layout()

    plt.savefig(f'Result/eval_fold_{fold}_{ext}.png')
    plt.show(block=False)


def plot_eval_measures_separate(val_mae, val_pcc, val_js, val_avg_pc, val_avg_bc, val_avg_ec, precision1=4, precision2=6, label_size1=10, label_size2=6):
    """
    Plots the evaluation measures for each fold separately

    Parameters:
    - val_mae(list): the MAE
    - val_pcc(list): the PCC
    - val_js(list): the JSD
    - val_avg_pc(list): the average PC
    - val_avg_bc(list): the average BC
    - val_avg_ec(list): the average EC
    - precision1(int): the precision
    - precision2(int): the precision
    - label_size1(int): the label size
    - label_size2(int): the label size

    Returns:
    - None
    """
    data1 = np.array([val_mae, val_pcc, val_js])
    data2 = np.array([val_avg_pc, val_avg_bc, val_avg_ec])
    measure_names1 = ['MAE', 'PCC', 'JSD']
    measure_names2 = ['MAE (PC)', 'MAE (EC)', 'MAE (BC)']

    def plot_data(data, measure_names, precision, title, file_name , label_size):
        barWidth = 0.25
        r1 = np.arange(len(data))
        r2 = [x + barWidth for x in r1]
        r3 = [x + barWidth for x in r2]

        plt.figure(figsize=(7, 4))

        # for each fold, plot the evaluation measures
        for i in range(data.shape[1]):
            plt.bar(r1, data[:, i], width=barWidth, edgecolor='grey', label=f'Fold {i}')

            # Add text on the bars
            for j in range(len(r1)):
                plt.text(r1[j], data[j, i] + data[j, i]*0.01, f'{data[j, i]:.{precision}f}', ha='center', fontsize=label_size)

            r1 = [x + barWidth for x in r1]

        plt.xlabel('Evaluation Measures', fontweight='bold')
        plt.xticks([r + barWidth for r in range(len(data))], measure_names)
        plt.ylabel('Values')
        plt.title(title)

        plt.legend()
        plt.tight_layout()
        plt.savefig(f'Result/{file_name}.png')
        plt.show()

    plot_data(data1, measure_names1, precision1, 'Evaluation Measures (MAE, PCC, JSD)', 'eval_measures1', label_size1)
    plot_data(data2, measure_names2, precision2, 'Evaluation Measures (MAE (PC), MAE (EC), MAE (BC))', 'eval_measures2', label_size2)

# Evaluate

In [ ]:
def evaluate(pred_matrices, gt_matrices):
    """
    Evaluates the model

    Parameters:
    - pred_matrices(np.array): the predicted matrices
    - gt_matrices(np.array): the ground truth matrices

    Returns:
    - mae(float): the MAE
    - pcc(float): the PCC
    - js_dis(float): the JSD
    - avg_mae_pc(float): the average MAE PC
    - avg_mae_ec(float): the average MAE EC
    - avg_mae_bc(float): the average MAE BC
    """
    num_test_samples = pred_matrices.shape[0]

    # post-processing
    pred_matrices[pred_matrices < 0] = 0
    gt_matrices[gt_matrices < 0] = 0

    # Initialize lists to store MAEs for each centrality measure
    mae_bc = []
    mae_ec = []
    mae_pc = []

    pred_1d_list = []
    gt_1d_list = []

    # Iterate over each test sample
    for i in range(num_test_samples):
        print(f'On sample {i}')

        # Convert adjacency matrices to NetworkX graphs
        pred_graph = nx.from_numpy_array(pred_matrices[i], edge_attr="weight")
        gt_graph = nx.from_numpy_array(gt_matrices[i], edge_attr="weight")

        # Compute centrality measures
        pred_bc = nx.betweenness_centrality(pred_graph, weight="weight")
        pred_ec = nx.eigenvector_centrality(pred_graph, weight="weight")
        pred_pc = nx.pagerank(pred_graph, weight="weight")

        gt_bc = nx.betweenness_centrality(gt_graph, weight="weight")
        gt_ec = nx.eigenvector_centrality(gt_graph, weight="weight")
        gt_pc = nx.pagerank(gt_graph, weight="weight")

        # Convert centrality dictionaries to lists
        pred_bc_values = list(pred_bc.values())
        pred_ec_values = list(pred_ec.values())
        pred_pc_values = list(pred_pc.values())

        gt_bc_values = list(gt_bc.values())
        gt_ec_values = list(gt_ec.values())
        gt_pc_values = list(gt_pc.values())

        # Compute MAEs
        mae_bc.append(mean_absolute_error(pred_bc_values, gt_bc_values))
        mae_ec.append(mean_absolute_error(pred_ec_values, gt_ec_values))
        mae_pc.append(mean_absolute_error(pred_pc_values, gt_pc_values))

        # Vectorize matrices
        pred_1d_list.append(MatrixVectorizer.vectorize(pred_matrices[i]))
        gt_1d_list.append(MatrixVectorizer.vectorize(gt_matrices[i]))

    # Compute average MAEs
    avg_mae_bc = sum(mae_bc) / len(mae_bc)
    avg_mae_ec = sum(mae_ec) / len(mae_ec)
    avg_mae_pc = sum(mae_pc) / len(mae_pc)

    # Concatenate flattened matrices
    pred_1d = np.concatenate(pred_1d_list)
    gt_1d = np.concatenate(gt_1d_list)

    # Compute metrics
    mae = mean_absolute_error(pred_1d, gt_1d)
    pcc = pearsonr(pred_1d, gt_1d)[0]
    js_dis = jensenshannon(pred_1d, gt_1d)

    print("MAE: ", mae)
    print("PCC: ", pcc)
    print("Jensen-Shannon Distance: ", js_dis)
    print("Average MAE betweenness centrality:", avg_mae_bc)
    print("Average MAE eigenvector centrality:", avg_mae_ec)
    print("Average MAE PageRank centrality:", avg_mae_pc)

    return mae, pcc, js_dis, avg_mae_pc, avg_mae_ec, avg_mae_bc



evaluation for paper

In [ ]:
!pip uninstall -y community
!pip uninstall -y python-louvain
!pip install python-louvain
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import mean_absolute_error
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import community as community_louvain
import os

def calculate_centralities(adj_matrix):
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError(f"Adjacency matrix is not square: shape={adj_matrix.shape}")
    print(f"Processing adjacency matrix of shape: {adj_matrix.shape}")

    G = nx.from_numpy_array(adj_matrix)
    partition = community_louvain.best_partition(G)

    # Calculate the participation coefficient with the partition
    pc_dict = participation_coefficient(G, partition)

    # Calculate averages of centrality measures
    pr = nx.pagerank(G, alpha=0.9)
    ec = nx.eigenvector_centrality_numpy(G, max_iter=100)
    bc = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    ns = np.array(list(nx.degree_centrality(G).values())) * (len(G.nodes()) - 1)
    acc = nx.average_clustering(G, weight=None)

    # Average participation coefficient
    pc_avg = np.mean(list(pc_dict.values()))

    return {
        'pr': np.mean(list(pr.values())),
        'ec': np.mean(list(ec.values())),
        'bc': np.mean(list(bc.values())),
        'ns': ns,
        'pc': pc_avg,
        'acc': acc
    }

def participation_coefficient(G, partition):
    # Initialize dictionary for participation coefficients
    pc_dict = {}

    # Calculate participation coefficient for each node
    for node in G.nodes():
        node_degree = G.degree(node)
        if node_degree == 0:
            pc_dict[node] = 0.0
        else:
            # Count within-module connections
            within_module_degree = sum(1 for neighbor in G[node] if partition[neighbor] == partition[node])
            # Calculate participation coefficient
            pc_dict[node] = 1 - (within_module_degree / node_degree) ** 2

    return pc_dict


def evaluate_all(true_hr_matrices, predicted_hr_matrices, output_path='26-randomCV.csv'):
    print(true_hr_matrices.shape)
    print(predicted_hr_matrices.shape)

    num_subjects = true_hr_matrices.shape[0]
    results = []

    for i in range(num_subjects):
        true_matrix = true_hr_matrices[i, :, :]
        pred_matrix = predicted_hr_matrices[i, :, :]

        print(f"Evaluating subject {i+1} with matrix shapes: true={true_matrix.shape}, pred={pred_matrix.shape}")

        if true_matrix.shape != pred_matrix.shape or true_matrix.shape[0] != true_matrix.shape[1]:
            print(f"Error: Matrix shape mismatch or not square for subject {i+1}: true={true_matrix.shape}, pred={pred_matrix.shape}")
            continue

        metrics = {
            'ID': i + 1,
            'MAE': mean_absolute_error(true_matrix.flatten(), pred_matrix.flatten()),
            'PCC': pearsonr(true_matrix.flatten(), pred_matrix.flatten())[0],
            'JSD': jensenshannon(true_matrix.flatten(), pred_matrix.flatten()),
        }

        true_metrics = calculate_centralities(true_matrix)
        pred_metrics = calculate_centralities(pred_matrix)

        for key in ['NS', 'PR', 'EC', 'BC', 'PC', 'ACC']:
            metrics[f'MAE in {key}'] = mean_absolute_error([true_metrics[key.lower()]], [pred_metrics[key.lower()]])

        results.append(metrics)

    df = pd.DataFrame(results)
    if not df.empty:
        # Check if the file exists to decide whether to write headers
        file_exists = os.path.isfile(output_path)

        df.to_csv(output_path, mode='a', header=not file_exists, index=False)
        print(f"Results appended to {output_path}.")
    else:
        print("No data to save.")


# Train Model

## Def Train and Test

In [ ]:
# Custom MSLE Loss function - Mean Squared Logarithmic Error Loss
class MSLELoss(nn.Module):
    """
    The Mean Squared Logarithmic Error (MSLE) loss function
    """
    def __init__(self):
        super(MSLELoss, self).__init__()

    def forward(self, inputs, targets):
        """
        Apply the Mean Squared Logarithmic Error (MSLE) between `predicted` and `actual`.

        Parameters:
        predicted (torch.Tensor): The predicted tensor.
        actual (torch.Tensor): The actual tensor.

        Returns:
        torch.Tensor: The MSLE loss.
        """
        # To ensure numerical stability, add a small constant (epsilon) before taking the log.
        inputs = torch.clamp(inputs, min=1e-7)
        targets = torch.clamp(targets, min=1e-7)

        # Calculate the squared logarithmic difference
        loss = torch.square(torch.log(inputs + 1) - torch.log(targets + 1))

        # Return the mean of the squared logarithmic error
        return torch.mean(loss)

In [ ]:
# criterion = nn.MSELoss()
criterion = MSLELoss()

In [ ]:
def train(model, optimizer, subjects_adj, subjects_labels, device, args, val_adj, val_ground_truth, total_epoch):
    """
    Trains the model

    Parameters:
    - model(nn.Module): the model
    - optimizer(torch.optim): the optimizer
    - subjects_adj(np.array): the training data (adjacency matrix)
    - subjects_labels(np.array): the training data (labels)
    - device(torch.device): the device, either 'cpu' or 'cuda'
    - args: the arguments

    Returns:
    - all_epochs_loss(list): the loss
    """
    all_epochs_loss = []
    val_epochs_loss = []
    no_epochs = args.epochs
    model.to(device)

    # Early stopping parameters
    early_stopping_patience = 10
    early_stopping_min_delta = 0.0001
    min_training_epochs = 100

    # Early stopping variables
    best_loss = float('inf')
    epochs_no_improve = 0
    early_stop = False
    epoch = 0

    while epoch < min_training_epochs or not early_stop:
    # while epoch < total_epoch:
        # Initialize lists to store loss and error for each epoch
        epoch_loss = []
        epoch_error = []

        model.train()

        for lr, hr in zip(subjects_adj, subjects_labels):
            optimizer.zero_grad()

            lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
            hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)

            model_outputs, net_outs, start_gcn_outs, layer_outs = model(lr)
            model_outputs = unpad(model_outputs, args.padding)

            padded_hr = pad_HR_adj(hr.cpu(), args.padding)
            _, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')
            U_hr = U_hr.to(device)

            loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights, U_hr) + criterion(
                model_outputs, hr)

            error = criterion(model_outputs, hr)

            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.item())
            epoch_error.append(error.item())

        mean_epoch_loss = np.mean(epoch_loss)
        all_epochs_loss.append(np.mean(epoch_loss))

        model.eval()
        with torch.no_grad():
            val_loss = []
            for lr, hr in zip(val_adj, val_ground_truth):
                lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)

                model_outputs, net_outs, start_gcn_outs, layer_outs = model(lr)
                model_outputs = unpad(model_outputs, args.padding)

                padded_hr = pad_HR_adj(hr.cpu(), args.padding)
                _, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')
                U_hr = U_hr.to(device)

                loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights, U_hr) + criterion(
                    model_outputs, hr)

                val_loss.append(loss.item())

            mean_val_loss = np.mean(val_loss)
            val_epochs_loss.append(mean_val_loss)

        if mean_val_loss < best_loss - early_stopping_min_delta:
            best_loss = mean_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience and epoch >= min_training_epochs:
            early_stop = True

        epoch += 1

        print(f'Epoch {epoch+1}/{no_epochs}, Training Loss: {np.mean(epoch_loss):.4f}, Error: {np.mean(epoch_error):.4f}, Val Loss: {mean_val_loss}')

    return all_epochs_loss, val_epochs_loss
    # return all_epochs_loss



def test(model, test_adj, test_labels, device, args):
    """
    Tests the model

    Parameters:
    - model(nn.Module): the model
    - test_adj(np.array): the testing data (adjacency matrix)
    - test_labels(np.array): the testing data (labels)
    - device(torch.device): the device, either 'cpu' or 'cuda'
    - args: the arguments

    Returns:
    - mae(float): the MAE
    - pcc(float): the PCC
    - js_dis(float): the JSD
    - avg_mae_pc(float): the average MAE PC
    - avg_mae_ec(float): the average MAE EC
    - avg_mae_bc(float): the average MAE BC
    """
    model.to(device)
    preds_list = []
    g_t = []

    i = 0
    # TESTING
    print("-------- Validation --------")
    for lr, hr in zip(test_adj, test_labels):

        all_zeros_lr = not np.any(lr)
        all_zeros_hr = not np.any(hr)

        if all_zeros_lr == False and all_zeros_hr == False:
            lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
            np.fill_diagonal(hr, 1)
            hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)
            preds, a, b, c = model(lr)
            preds = unpad(preds, args.padding)

            preds_list.append(preds.cpu().detach().numpy())

            g_t.append(hr.cpu().detach().numpy())
            i += 1
    pred_mat = np.array(preds_list)
    gt_mat = np.array(g_t)
    mae, pcc, js_dis, avg_mae_pc, avg_mae_ec, avg_mae_bc = evaluate(pred_mat, gt_mat)
    return pred_mat, mae, pcc, js_dis, avg_mae_pc, avg_mae_ec, avg_mae_bc


## hyperparameters

In [ ]:
parser = argparse.ArgumentParser(description='GSR-Net')
parser.add_argument('--epochs', type=int, default=500, metavar='no_epochs',
                    help='number of episode to train ')
parser.add_argument('--lr', type=float, default=0.0001, metavar='lr',
                    help='learning rate (default: 0.0001 using Adam Optimizer)')
parser.add_argument('--splits', type=int, default=3, metavar='n_splits',
                    help='no of cross validation folds')
parser.add_argument('--lmbda', type=int, default=16, metavar='L',
                    help='self-reconstruction error hyperparameter')
parser.add_argument('--lr_dim', type=int, default=160, metavar='N',
                    help='adjacency matrix input dimensions')
parser.add_argument('--hr_dim', type=int, default=320, metavar='N',
                    help='super-resolved adjacency matrix output dimensions')
parser.add_argument('--hidden_dim', type=int, default=320, metavar='N',
                    help='hidden GraphConvolutional layer dimensions')
parser.add_argument('--padding', type=int, default=26, metavar='padding',
                    help='dimensions of padding')
parser.add_argument('--num_sampled_nodes', type=int, default=100, metavar='num_sampled_nodes',
                        help='number of sampled nodes')
args, unknown = parser.parse_known_args()

## Load Data and Start Training

In [ ]:
subjects_adj_split_1, subjects_gt_split_1, subjects_adj_split_2, subjects_gt_split_2, subjects_adj_split_3, subjects_gt_split_3, test_adj, _ = data()

def data_es(data_lr_reshaped, data_hr_reshaped):
    # use the last 20 samples for early stopping
    num_samples = data_lr_reshaped.shape[0]
    train_size = int(num_samples - 20)

    subjects_adj = data_lr_reshaped[:train_size]
    subjects_labels = data_hr_reshaped[:train_size]

    test_adj = data_lr_reshaped[train_size:]
    test_labels = data_hr_reshaped[train_size:]

    return subjects_adj, subjects_labels, test_adj, test_labels

train_subjects_adj_split_1, train_subjects_labels_split_1, test_adj_split_1, test_labels_split_1 = data_es(
    subjects_adj_split_1, subjects_gt_split_1
)
train_subjects_adj_split_2, train_subjects_labels_split_2, test_adj_split_2, test_labels_split_2 = data_es(
    subjects_adj_split_2, subjects_gt_split_2
)
train_subjects_adj_split_3, train_subjects_labels_split_3, test_adj_split_3, test_labels_split_3 = data_es(
    subjects_adj_split_3, subjects_gt_split_3
)


# Perform 3-fold validation
splits = [
    (0, 1, 2),
    (0, 2, 1),
    (1, 2, 0)
]

print("Torch:")

ks = [0.9, 0.7, 0.6, 0.5]

In [ ]:
i = 0
# to cuda
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print('Device: ', device)

if not os.path.exists('./Result'):
    os.mkdir('./Result/')

In [ ]:
STANDARD_MEASURE_NAMES = ['MAE', 'PCC', 'JSD']
MAE_MEASURE_NAMES = ['MAE (PC)', 'MAE (EC)', 'MAE (BC)']
all_train_loss = []
val_mae = []
val_pcc = []
val_js = []
val_avg_pc = []
val_avg_ec = []
val_avg_bc = []
for train_indices in splits:
    model = GSRNet(ks, args)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    total_epoch = 0

    print('======== Fold ', i, '========')
    if i == 0:
        total_epoch = 130
        subjects_adj, eval_adj, subjects_gt, eval_gt, val_adj, val_gt= (

            np.concatenate((subjects_adj_split_1, subjects_adj_split_2), axis=0),
            subjects_adj_split_3,
            np.concatenate((subjects_gt_split_1, subjects_gt_split_2), axis=0),
            subjects_gt_split_3,
            np.concatenate((test_adj_split_1, test_adj_split_2), axis=0),
            np.concatenate((test_labels_split_1, test_labels_split_2), axis=0),
        )
    elif i == 1:
        total_epoch = 126
        subjects_adj, eval_adj, subjects_gt, eval_gt, val_adj, val_gt= (

            np.concatenate((subjects_adj_split_1, subjects_adj_split_3), axis=0),
            subjects_adj_split_2,
            np.concatenate((subjects_gt_split_1, subjects_gt_split_3), axis=0),
            subjects_gt_split_2,
            np.concatenate((test_adj_split_1, test_adj_split_3), axis=0),
            np.concatenate((test_labels_split_1, test_labels_split_3), axis=0),
        )
    elif i == 2:
        total_epoch = 127
        subjects_adj, eval_adj, subjects_gt, eval_gt, val_adj, val_gt= (

            np.concatenate((subjects_adj_split_2, subjects_adj_split_3), axis=0),
            subjects_adj_split_1,
            np.concatenate((subjects_gt_split_2, subjects_gt_split_3), axis=0),
            subjects_gt_split_1,
            np.concatenate((test_adj_split_2, test_adj_split_3), axis=0),
            np.concatenate((test_labels_split_2, test_labels_split_3), axis=0),
        )

    epoch_loss = train(model, optimizer, subjects_adj, subjects_gt, device, args)
    epoch_loss, val_epochs_loss = train(model, optimizer, subjects_adj, subjects_gt, device, args, val_adj, val_gt, total_epoch=total_epoch)

    all_train_loss.append(epoch_loss)

    epoch_loss_df = pd.DataFrame({'epoch': list(range(1, len(epoch_loss) + 1)), 'loss': epoch_loss})
    epoch_loss_df.to_csv(f'26_fold_{i}_loss_curve.csv', index=False)

    val_epoch_loss_df = pd.DataFrame({'epoch': list(range(1, len(val_epochs_loss) + 1)), 'loss': val_epochs_loss})
    val_epoch_loss_df.to_csv(f'26_fold_{i}_val_loss_curve.csv', index=False)

    # save the fold model
    torch.save(model, f'./Result/model_fold_{i}.pth')

    preds, mae, pcc, js_dis, avg_mae_pc, avg_mae_ec, avg_mae_bc = test(model, eval_adj, eval_gt, device, args)
    val_mae.append(mae)
    val_pcc.append(pcc)
    val_js.append(js_dis)
    val_avg_pc.append(avg_mae_pc)
    val_avg_ec.append(avg_mae_ec)
    val_avg_bc.append(avg_mae_bc)

    evaluate_all(eval_gt, preds, output_path=f'26_fold_{i}_evaluation.csv')

    plot_fold_eval_measures([mae, pcc, js_dis], i+1, STANDARD_MEASURE_NAMES, 'standard')
    plot_fold_eval_measures([avg_mae_pc, avg_mae_ec, avg_mae_bc], i+1, MAE_MEASURE_NAMES, 'mae')
    i += 1

## Save and Visualization

In [ ]:
# save the loss and mae
all_train_loss = np.array(all_train_loss)
val_mae = np.array(val_mae)
val_pcc = np.array(val_pcc)
val_js = np.array(val_js)
val_avg_pc = np.array(val_avg_pc)
val_avg_bc = np.array(val_avg_bc)
val_avg_ec = np.array(val_avg_ec)


In [ ]:
# plot the training loss
plot_loss(all_train_loss)

# Predict

In [ ]:
def predict(model, test_adj, device, args):
    """
    Predicts the data

    Parameters:
    - model(nn.Module): the model
    - test_adj(np.array): the testing data (adjacency matrix)
    - device(torch.device): the device, either 'cpu' or 'cuda'
    - args: the arguments

    Returns:
    - preds_list(list): the predictions
    """
    preds_list = []
    i = 0
    for lr in test_adj:
        lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
        preds, a, b, c = model(lr)
        preds = unpad(preds, args.padding)
        preds_list.append(preds.cpu().detach().numpy())
        i += 1
    return preds_list

In [ ]:
# predict
vectorizer = MatrixVectorizer()
for n in range(args.splits):
    model = torch.load(f'./Result/model_fold_{n}.pth')
    preds_list = predict(model, test_adj, device, args)
    for i in range(len(preds_list)):
        # vectorize
        preds_list[i] = vectorizer.vectorize(preds_list[i])
    # save the predictions
    preds_arr = np.array(preds_list)
    preds = preds_arr.flatten()

    with open(f'Result/predictions_fold_{n}.csv', 'w') as f:
        f.write("Id,Predicted\n")
        for i, pred in enumerate(preds):
            f.write(f"{i + 1},{pred}\n")